# Create Additional Features

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import repeatable_functions as rf
import warnings

# Ignore PerformanceWarning and UserWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

### Load Datasets

First we can load our match and player datasets:

In [ ]:
# Load datasets
match_df = pd.read_csv('datasets\match_df.csv')
player_df = pd.read_csv('datasets\player_df.csv')

### Function to join player and match datasets

We can finally merge the match and player datasets, using an inner join to remove missing matches:

In [ ]:
def create_player_match_df(match_df, player_df):
    """
    Merges match data with player statistics to create a comprehensive match-level DataFrame.

    Parameters:
    - match_df (pd.DataFrame): DataFrame containing match data with 'player' and 'opponent' columns.
    - player_df (pd.DataFrame): DataFrame containing player statistics with formatted names.

    Returns:
    - player_match_df (pd.DataFrame): A merged DataFrame with player and opponent statistics included.

    Notes:
    - First, player statistics are merged onto 'player' using an inner join.
    - Then, opponent statistics are merged onto 'opponent' using another inner join.
    - Column names are prefixed with 'player_' and 'opp_' to differentiate between players.
    - Uses inner joins to ensure that only matches with available player data are included.
    """

    # Merge match data with player statistics for the main player
    first_join_df = pd.merge(
        match_df,
        player_df.add_prefix('player_'),  # Add 'player_' prefix to differentiate player stats
        how='inner',
        left_on='player',
        right_on='player_formatted_name'  # Match on formatted player names
    )

    # Merge opponent statistics onto the newly merged DataFrame
    player_match_df = pd.merge(
        first_join_df,
        player_df.add_prefix('opp_'),  # Add 'opp_' prefix to differentiate opponent stats
        how='inner',
        left_on='opponent',
        right_on='opp_formatted_name'  # Match on formatted opponent names
    )

    return player_match_df


In [206]:
# Run the function 
player_match_df = create_player_match_df(match_df, player_df)

### Function to filter player stats only by the previous year to the match

Since we have up-to-date player stat info, and some of the matches took place before the stats were collected, we will engineer features that show all the stats for the previous year of the match and drop all other yearly stats:

In [207]:
def filter_by_previous_year_columns(df):
    """
    Filters and processes year-specific columns in a DataFrame to retain only the previous year's data.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing year-specific columns.

    Returns:
    - df_new (pd.DataFrame): A DataFrame where year-specific columns are replaced with '_previous_year' equivalents.

    Notes:
    - Identifies columns that have a four-digit year suffix (e.g., '_2023').
    - Creates new columns with '_previous_year' suffix, initialized as None.
    - Populates '_previous_year' columns based on the current row's 'year' value.
    - Drops the original year-specific columns after transformation.
    - Computes 'vsOther' metrics using wins and Top100 match data.
    """

    df_new = df.copy()
    
    # Identify columns with a 4-digit year suffix between 2000 and 2025
    year_columns = [col for col in df_new.columns if col[-4:].isdigit() and 2003 <= int(col[-4:]) <= 2025]
    
    # Map of original year-specific columns to new '_previous_year' columns
    previous_year_cols = {col: col.replace(col[-4:], '_previous_year') for col in year_columns}

    # Pre-fill new columns with None
    for new_col in previous_year_cols.values():
        df_new[new_col] = None

    # Process each year-specific column
    for col, new_col in previous_year_cols.items():
        year_suffix = int(col[-4:])
        matching_rows = df_new['year'] == (year_suffix + 1)
        df_new.loc[matching_rows, new_col] = df_new.loc[matching_rows, col]

    # Drop the original year-specific columns
    df_new.drop(columns=year_columns, inplace=True)

    # Calculate 'vsOther' metrics
    for role in ['player', 'opp']:
        df_new[f'{role}_vsOther_wins__previous_year'] = (
            df_new[f'{role}_wins__previous_year'] - df_new[f'{role}_vsTop100_wins__previous_year']
        )
        df_new[f'{role}_vsOther_losses__previous_year'] = (
            df_new[f'{role}_losses__previous_year'] - df_new[f'{role}_vsTop100_losses__previous_year']
        )

    return df_new

In [208]:
player_match_df = filter_by_previous_year_columns(player_match_df)

### Function to show previous year stats for matching conditions

We can also then create additional features for the number of wins in previous year for the same series, surface, round and category of opponent as the current match. We can also then add features for the win rates, adding more weight to players who have played more games:

In [ ]:
def match_match_details_to_stats(df):
    """
    Maps match-level details to player statistics from the previous year and calculates logarithmic weighted win rates.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing match details and player statistics.

    Returns:
    - df_new (pd.DataFrame): A DataFrame with new mapped columns and calculated win rates.

    Notes:
    - Creates columns that map previous-year statistics based on match details (e.g., surface, round, series).
    - Computes logarithmic weighted win rates for players and opponents.
    - Drops unnecessary previous-year statistics after mapping.
    """

    # Create a copy of the DataFrame to avoid modifying the original
    df_new = df.copy()

    # Function to create and map columns dynamically
    def create_and_map_columns(row):
        new_columns_map = {
            'player_current_series_wins__previous_year': f"player_{row['Series']}_wins__previous_year",
            'player_current_series_losses__previous_year': f"player_{row['Series']}_losses__previous_year",
            'player_current_surface_wins__previous_year': f"player_{row['Surface']}_wins__previous_year",
            'player_current_surface_losses__previous_year': f"player_{row['Surface']}_losses__previous_year",
            'player_current_round_wins__previous_year': f"player_{row['Round']}_wins__previous_year",
            'player_current_round_losses__previous_year': f"player_{row['Round']}_losses__previous_year",
            'player_current_vsTop_wins__previous_year': f"player_{row['player_vs_top']}_wins__previous_year",
            'player_current_vsTop_losses__previous_year': f"player_{row['player_vs_top']}_losses__previous_year",
            'opp_current_series_wins__previous_year': f"opp_{row['Series']}_wins__previous_year",
            'opp_current_series_losses__previous_year': f"opp_{row['Series']}_losses__previous_year",
            'opp_current_surface_wins__previous_year': f"opp_{row['Surface']}_wins__previous_year",
            'opp_current_surface_losses__previous_year': f"opp_{row['Surface']}_losses__previous_year",
            'opp_current_round_wins__previous_year': f"opp_{row['Round']}_wins__previous_year",
            'opp_current_round_losses__previous_year': f"opp_{row['Round']}_losses__previous_year",
            'opp_current_vsTop_wins__previous_year': f"opp_{row['opp_vs_top']}_wins__previous_year",
            'opp_current_vsTop_losses__previous_year': f"opp_{row['opp_vs_top']}_losses__previous_year",
        }
        
        # Assign values from corresponding previous-year columns if they exist, otherwise set to None
        for new_col, orig_col in new_columns_map.items():
            row[new_col] = row.get(orig_col, None)
        return row

    # Apply function row-wise to create mapped columns
    df_new = df_new.apply(create_and_map_columns, axis=1)

    # Function to calculate logarithmic weighted win rates
    def calculate_log_weighted_win_rates(row):
        categories = ['series', 'surface', 'round', 'vsTop']
        for category in categories:
            # Player calculations
            wins_col = f'player_current_{category}_wins__previous_year'
            losses_col = f'player_current_{category}_losses__previous_year'
            win_rate_col = f'player_current_{category}_win_rate__previous_year'
            
            wins = row.get(wins_col, 0)
            losses = row.get(losses_col, 0)
            total_games = wins + losses

            row[win_rate_col] = (wins / total_games) * np.log(total_games + 1) if total_games > 0 else 0

            # Opponent calculations
            opp_wins_col = f'opp_current_{category}_wins__previous_year'
            opp_losses_col = f'opp_current_{category}_losses__previous_year'
            opp_win_rate_col = f'opp_current_{category}_win_rate__previous_year'

            opp_wins = row.get(opp_wins_col, 0)
            opp_losses = row.get(opp_losses_col, 0)
            total_opp_games = opp_wins + opp_losses

            row[opp_win_rate_col] = (opp_wins / total_opp_games) * np.log(total_opp_games + 1) if total_opp_games > 0 else 0

        return row

    # Apply function row-wise to calculate win rates
    df_new = df_new.apply(calculate_log_weighted_win_rates, axis=1)

    # Drop original previous-year columns after processing
    columns_to_drop = [
        col for col in df_new.columns 
        if col.startswith(('player_', 'opp_')) and 
        ('_wins_' in col or '_losses_' in col) and 
        ('__previous_year' in col) and 
        ('current' not in col) and 
        col not in ['player_wins__previous_year', 'player_losses__previous_year']
    ]

    df_new.drop(columns=columns_to_drop, inplace=True)

    return df_new


In [210]:
# Run the function
player_match_df = match_match_details_to_stats(player_match_df)

We can run our previous function showing the top null columns on our new player_match_df:

In [ ]:
rf.show_columns_with_most_nulls(player_match_df)


Top 10 columns with the most null values:



,Null Count,Null Percentage
opp_current_series_wins__previous_year,14854,15.601794
opp_current_series_losses__previous_year,14854,15.601794
player_current_series_wins__previous_year,14700,15.440041
player_current_series_losses__previous_year,14700,15.440041
opp_current_surface_wins__previous_year,3890,4.085834
opp_current_surface_losses__previous_year,3890,4.085834
player_current_surface_losses__previous_year,3702,3.888370
player_current_surface_wins__previous_year,3702,3.888370
opp_bps_faced_pct__previous_year,3515,3.691955
opp_bps_saved_pct__previous_year,3515,3.691955


Given that the null values of these columns are just a small percentage, we can assume that missing data for the previous year is simply due to the opposition not having stats for the previous year and we can therefore simply fill these values with zero: 

In [212]:
player_match_df.loc[:, player_match_df.columns.str.contains('previous_year')] = player_match_df.loc[:, player_match_df.columns.str.contains('previous_year')].fillna(0)

### Function to add features showing difference in player and opposition stats

Now we've filled the missing values with zero, we can engineer additional features based on the difference between the player's and the opposition's previous year stats:

In [213]:
def add_diff_columns(df):
    """
    Adds new columns to the DataFrame representing the difference between player and opponent 
    statistics from the previous year.

    The function:
    - Identifies all player-related columns ending with '_previous_year'.
    - Matches them with corresponding opponent columns (with prefix 'opp_').
    - Creates new columns with the suffix '_diff', representing the difference between the 
      player's and opponent's values for each matched statistic.

    Parameters:
        df (pandas.DataFrame): A DataFrame containing columns prefixed with 'player_' and 'opp_', 
                               specifically for previous year statistics.

    Returns:
        pandas.DataFrame: The input DataFrame with additional '_diff' columns indicating the 
                          player-opponent differences.
    """
    # Get all column names
    cols = df.columns

    # Extract stat names without prefix
    stat_suffixes = set()
    for col in cols:
        if col.startswith("player_") and col.endswith("_previous_year"):
            suffix = col.replace("player_", "")
            if f"opp_{suffix}" in cols:
                stat_suffixes.add(suffix)

    # Create diff columns
    for suffix in stat_suffixes:
        player_col = f"player_{suffix}"
        opp_col = f"opp_{suffix}"
        diff_col = f"{suffix}_diff"
        df[diff_col] = df[player_col] - df[opp_col]

    return df


In [214]:
# Run the function
player_match_df = add_diff_columns(player_match_df)

### Function to create cumulative score features

We can now create a function to engineer additional features summing the scores from the previous 2 to 10 matches:

In [ ]:
def create_prior_score_features(df):
    """
    Creates cumulative score features over the past 2 to 10 matches for various scoring metrics,
    including standard and straight-sets variations. Drops original per-match and base score columns
    after aggregation.

    Parameters:
        df (pandas.DataFrame): DataFrame containing past match score features.

    Returns:
        pandas.DataFrame: Modified DataFrame with cumulative features added and redundant columns removed.
    """
    df_new = df.copy()

    score_types = ['rank_score', 'pts_score', 'ss_rank_score', 'ss_pts_score']

    for score in score_types:
        for n in range(2, 11):
            cols = [f"{score}_{i}_match_ago" for i in range(1, n + 1)]
            existing_cols = [col for col in cols if col in df_new.columns]
            if existing_cols:  # Avoid issues if some columns are missing
                df_new[f'{score}_past{n}'] = df_new[existing_cols].sum(axis=1)

    # Drop all per-match score columns
    columns_to_drop = [col for col in df_new.columns if 'ago' in col]
    df_new.drop(columns=columns_to_drop, inplace=True)

    # Drop additional raw score columns
    extra_columns = ['rank_score', 'pts_score', 'straight_sets', 'ss_rank_score', 'ss_pts_score']
    df_new.drop(columns=[col for col in extra_columns if col in df_new.columns], inplace=True)

    return df_new


In [216]:
# Run the function
player_match_df = create_prior_score_features(player_match_df)

In [ ]:
player_match_df2 = pd.read_csv('datasets\player_match_df_clean.csv')

### Function to remove any remaining rows with null values and redundant columns

Now that we have finished engineering our features, we can build a function to remove any remaining rows containing null values, and to remove any remaining columns that we don't require for model building:

In [ ]:
def clean_dataset(df):
    """
    Cleans the dataset by removing null values and dropping irrelevant columns.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing match data.

    Returns:
    - df_new (pd.DataFrame): A cleaned DataFrame with dropped null values and unnecessary columns.

    Notes:
    - Prints the initial number of rows before cleaning.
    - Drops rows with missing values.
    - Drops irrelevant columns such as player/opponent names and match details.
    - Prints the number of removed rows and the final number of rows after cleaning.
    """

    # Count the initial number of rows
    initial_count = len(df)
    print(f"Previous number of rows in match_df: {initial_count}")

    # Create a copy to avoid modifying the original DataFrame
    df_new = df.copy()

    # Drop null values
    df_new = df_new.dropna()

    # Drop irrelevant columns
    columns_to_drop = [
        'player', 'opponent', 'Series', 'Surface', 'Round', 'date', 
        'opp_formatted_name', 'opp_name', 'player_formatted_name', 
        'player_vs_top', 'opp_vs_top', 'player_name'
    ]
    
    df_new = df_new.drop(columns=columns_to_drop, errors='ignore')

    # Count the number of rows after removal
    final_count = len(df_new)

    # Calculate the number of removed rows
    removed_count = initial_count - final_count

    # Print summary statistics
    print(f"Number of removed rows: {removed_count}")
    print(f"New number of rows in match_df: {final_count}")  # Extra print statement
    print("Removed redundant columns")

    return df_new


In [219]:
# Run the function
player_match_df = clean_dataset(player_match_df)

Previous number of rows in match_df: 95207
Number of removed rows: 0
New number of rows in match_df: 95207
Removed redundant columns


We can now save our new player_match_df before we move onto feature selection:

In [ ]:
# Save to file
player_match_df.to_csv('datasets\player_match_df_clean.csv', index=False)